In [1]:
from miditok import MMM, MuMIDI, TokenizerConfig, Octuple, REMI
from miditok.pytorch_data import DatasetTok, DataCollator
from pathlib import Path
from symusic import Score
from torch.utils.data import Dataset
import torch
from tqdm import tqdm
import os

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Tokenizer 선언

In [41]:
from __future__ import annotations

from typing import TYPE_CHECKING

from symusic import (
    Note,
    Pedal,
    PitchBend,
    Score,
    Tempo,
    TimeSignature,
    Track,
)

from miditok.classes import Event, TokenizerConfig, TokSequence
from miditok.constants import MIDI_INSTRUMENTS, TIME_SIGNATURE
from miditok.midi_tokenizer import MIDITokenizer
from miditok.utils import compute_ticks_per_bar, compute_ticks_per_beat

if TYPE_CHECKING:
    from pathlib import Path

class CustomREMI(REMI):
    def _tokens_to_midi(
        self,
        tokens: TokSequence | list[TokSequence],
        programs: list[tuple[int, bool]] | None = None,
    ) -> Score:
        r"""
        Convert tokens (:class:`miditok.TokSequence`) into a MIDI.

        This is an internal method called by ``self.tokens_to_midi``, intended to be
        implemented by classes inheriting :class:`miditok.MidiTokenizer`.

        :param tokens: tokens to convert. Can be either a list of
            :class:`miditok.TokSequence` or a list of :class:`miditok.TokSequence`s.
        :param programs: programs of the tracks. If none is given, will default to
            piano, program 0. (default: ``None``)
        :return: the midi object (:class:`symusic.Score`).
        """
        # Unsqueeze tokens in case of one_token_stream
        if self.one_token_stream:  # ie single token seq
            tokens = [tokens]
        for i in range(len(tokens)):
            tokens[i] = tokens[i].tokens
        midi = Score(self.time_division)

        # RESULTS
        tracks: dict[int, Track] = {}
        tempo_changes, time_signature_changes = [], []

        def check_inst(prog: int) -> None:
            if prog not in tracks:
                tracks[prog] = Track(
                    program=0 if prog == -1 else prog,
                    is_drum=prog == -1,
                    name="Drums" if prog == -1 else MIDI_INSTRUMENTS[prog]["name"],
                )

        current_instrument = None
        for si, seq in enumerate(tokens):
            # First look for the first time signature if needed
            if si == 0:
                if self.config.use_time_signatures:
                    for token in seq:
                        tok_type, tok_val = token.split("_")
                        if tok_type == "TimeSig":
                            time_signature_changes.append(
                                TimeSignature(0, *list(map(int, tok_val.split("/"))))
                            )
                            break
                        if tok_type in [
                            "Pitch",
                            "PitchDrum",
                            "Velocity",
                            "Duration",
                            "PitchBend",
                            "Pedal",
                        ]:
                            break
                if len(time_signature_changes) == 0:
                    time_signature_changes.append(TimeSignature(0, *TIME_SIGNATURE))
            current_time_sig = time_signature_changes[-1]
            ticks_per_bar = compute_ticks_per_bar(
                current_time_sig, midi.ticks_per_quarter
            )
            ticks_per_beat = self._tpb_per_ts[current_time_sig.denominator]
            ticks_per_pos = ticks_per_beat // self.config.max_num_pos_per_beat

            # Set tracking variables
            current_tick = tick_at_last_ts_change = tick_at_current_bar = 0
            current_bar = -1
            bar_at_last_ts_change = 0
            current_program = 0
            previous_note_end = 0
            previous_pitch_onset = {prog: -128 for prog in self.config.programs}
            previous_pitch_chord = {prog: -128 for prog in self.config.programs}
            active_pedals = {}

            # Set track / sequence program if needed
            if not self.one_token_stream:
                is_drum = False
                if programs is not None:
                    current_program, is_drum = programs[si]
                current_instrument = Track(
                    program=current_program,
                    is_drum=is_drum,
                    name="Drums"
                    if current_program == -1
                    else MIDI_INSTRUMENTS[current_program]["name"],
                )

            # Decode tokens
            for ti, token in enumerate(seq):
                tok_type, tok_val = token.split("_")
                if tok_type == "Bar":
                    current_bar = int(tok_val)
                    if current_bar > 0:
                        current_tick = tick_at_current_bar + ticks_per_bar
                    tick_at_current_bar = current_tick
                elif tok_type == "Rest":
                    current_tick = max(previous_note_end, current_tick)
                    current_tick += self._tpb_rests_to_ticks[ticks_per_beat][tok_val]
                    real_current_bar = (
                        bar_at_last_ts_change
                        + (current_tick - tick_at_last_ts_change) // ticks_per_bar
                    )
                    if real_current_bar > current_bar:
                        # In case we instantly begin with a Rest,
                        # we need to update current_bar
                        if current_bar == -1:
                            current_bar = 0
                        tick_at_current_bar += (
                            real_current_bar - current_bar
                        ) * ticks_per_bar
                        current_bar = real_current_bar
                elif tok_type == "Position":
                    if current_bar == -1:
                        # as this Position token occurs before any Bar token
                        current_bar = 0
                    current_tick = tick_at_current_bar + int(tok_val) * ticks_per_pos
                elif tok_type in {
                    "Pitch",
                    "PitchDrum",
                    "PitchIntervalTime",
                    "PitchIntervalChord",
                }:
                    if tok_type in {"Pitch", "PitchDrum"}:
                        pitch = int(tok_val)
                        previous_pitch_onset[current_program] = pitch
                        previous_pitch_chord[current_program] = pitch
                    # We update previous_pitch_onset and previous_pitch_chord even if
                    # the try fails.
                    elif tok_type == "PitchIntervalTime":
                        pitch = previous_pitch_onset[current_program] + int(tok_val)
                        previous_pitch_onset[current_program] = pitch
                        previous_pitch_chord[current_program] = pitch
                    else:  # PitchIntervalChord
                        pitch = previous_pitch_chord[current_program] + int(tok_val)
                        previous_pitch_chord[current_program] = pitch
                    try:
                        vel_type, vel = seq[ti + 1].split("_")
                        dur_type, dur = seq[ti + 2].split("_")
                        if vel_type == "Velocity" and dur_type == "Duration":
                            dur = self._tpb_tokens_to_ticks[ticks_per_beat][dur]
                            new_note = Note(
                                current_tick,
                                dur,
                                pitch,
                                int(vel),
                            )
                            if self.one_token_stream:
                                check_inst(current_program)
                                tracks[current_program].notes.append(new_note)
                            else:
                                current_instrument.notes.append(new_note)
                            previous_note_end = max(
                                previous_note_end, current_tick + dur
                            )
                    except IndexError:
                        # A well constituted sequence should not raise an exception
                        # However with generated sequences this can happen, or if the
                        # sequence isn't finished
                        pass
                elif tok_type == "Program":
                    current_program = int(tok_val)
                elif tok_type == "Tempo":
                    if si == 0:
                        tempo_changes.append(Tempo(current_tick, float(tok_val)))
                    previous_note_end = max(previous_note_end, current_tick)
                elif tok_type == "TimeSig":
                    num, den = self._parse_token_time_signature(tok_val)
                    if (
                        num != current_time_sig.numerator
                        or den != current_time_sig.denominator
                    ):
                        current_time_sig = TimeSignature(current_tick, num, den)
                        if si == 0:
                            time_signature_changes.append(current_time_sig)
                        tick_at_last_ts_change = tick_at_current_bar  # == current_tick
                        bar_at_last_ts_change = current_bar
                        ticks_per_bar = compute_ticks_per_bar(
                            current_time_sig, midi.ticks_per_quarter
                        )
                        ticks_per_beat = self._tpb_per_ts[den]
                        ticks_per_pos = (
                            ticks_per_beat // self.config.max_num_pos_per_beat
                        )
                elif tok_type == "Pedal":
                    pedal_prog = (
                        int(tok_val) if self.config.use_programs else current_program
                    )
                    if self.config.sustain_pedal_duration and ti + 1 < len(seq):
                        if seq[ti + 1].split("_")[0] == "Duration":
                            duration = self._tpb_tokens_to_ticks[ticks_per_beat][
                                seq[ti + 1].split("_")[1]
                            ]
                            # Add instrument if it doesn't exist, can happen for the
                            # first tokens
                            new_pedal = Pedal(current_tick, duration)
                            if self.one_token_stream:
                                check_inst(pedal_prog)
                                tracks[pedal_prog].pedals.append(new_pedal)
                            else:
                                current_instrument.pedals.append(new_pedal)
                    elif pedal_prog not in active_pedals:
                        active_pedals[pedal_prog] = current_tick
                elif tok_type == "PedalOff":
                    pedal_prog = (
                        int(tok_val) if self.config.use_programs else current_program
                    )
                    if pedal_prog in active_pedals:
                        new_pedal = Pedal(
                            active_pedals[pedal_prog],
                            current_tick - active_pedals[pedal_prog],
                        )
                        if self.one_token_stream:
                            check_inst(pedal_prog)
                            tracks[pedal_prog].pedals.append(
                                Pedal(
                                    active_pedals[pedal_prog],
                                    current_tick - active_pedals[pedal_prog],
                                )
                            )
                        else:
                            current_instrument.pedals.append(new_pedal)
                        del active_pedals[pedal_prog]
                elif tok_type == "PitchBend":
                    new_pitch_bend = PitchBend(current_tick, int(tok_val))
                    if self.one_token_stream:
                        check_inst(current_program)
                        tracks[current_program].pitch_bends.append(new_pitch_bend)
                    else:
                        current_instrument.pitch_bends.append(new_pitch_bend)

            # Add current_inst to midi and handle notes still active
            if not self.one_token_stream:
                midi.tracks.append(current_instrument)

        # create MidiFile
        if self.one_token_stream:
            midi.tracks = list(tracks.values())
        midi.tempos = tempo_changes
        midi.time_signatures = time_signature_changes

        return midi

In [2]:
TOKENIZER_PARAMS = {
    "pitch_range": (0, 127),
    "num_velocities": 127,
    # "special_tokens": ["PAD", "BOS", "EOS", "MASK"],
    "max_bar_embedding" : 3000,
    "use_chords": True,
    "use_tempos": True,
    "use_programs": True,
    "num_tempos": 211,  # number of tempo bins
    "tempo_range": (40, 250),  # (min, max)
}
config = TokenizerConfig(**TOKENIZER_PARAMS)
TOKENIZER_NAME = REMI
# TOKENIZER_NAME = CustomREMI

tokenizer = TOKENIZER_NAME(config)
len(tokenizer)

770

### 음악 분할

In [46]:
### 노래가 저장되어있는 폴더 명
dir_path ='./clean_midi'
### 저장할 폴더
CHUNK_PATH = './lakh'

## 몇 바를 기준으로 나눌 것인가
bar_num = 3

for song in tqdm(os.listdir(dir_path)):
    # print(song)
    if not os.path.exists(f'{CHUNK_PATH}/{song[:-4]}'):
            os.makedirs(f'{CHUNK_PATH}/{song[:-4]}')
    tokens = tokenizer(f'{dir_path}/{song}')
    li=[]
    tempo =0
    for ids, token in enumerate(tokens.tokens):
        if ids == 0:
            li.append(token)
            continue
        
        if tokens.events[ids].type_ =='Tempo':
            tempo = token
            continue
            

        if tokens.events[ids].type_ =='Bar':
            li.append(tempo)
            current_bar = int(token[4:])
        
            if current_bar % bar_num == 0 :
                tokenizer(li).dump_midi(f'{CHUNK_PATH}/{song[:-4]}/{current_bar}.mid')
                li=[]
            li.append(token)
            continue
        
        li.append(token)
    
    li.append(tempo)
    tokenizer(li).dump_midi(f'{CHUNK_PATH}/{song[:-4]}/{current_bar+1}.mid')


100%|██████████| 364/364 [02:19<00:00,  2.61it/s]


### tokenizer 갯수로 bar 나누기

In [8]:
### 노래가 저장되어있는 폴더 명
dir_path ='./clean_midi'
### 저장할 폴더
CHUNK_PATH = './lakh_max_token'

## 최대 토크나이저를 몇으로 설정할 건가
# 1000개 로 정해도 내부함수에 의해서 1000개 보다 많은 수가 나옴 적당히 조절
max_tokenizer = 1000

for in_path in tqdm(os.listdir(dir_path)):
    # print(song)
    for song in (os.listdir(f'{dir_path}/{in_path}')):
        if not os.path.exists(f'{CHUNK_PATH}/{song[:-4]}'):
                os.makedirs(f'{CHUNK_PATH}/{song[:-4]}')
        else:
            continue
        try:
            tokens = tokenizer(f'{dir_path}/{in_path}/{song}')
        except:
            continue

        li=[]

        temporary = []

        tempo =0
        num_mid = 0

        for ids, token in enumerate(tokens.tokens):
            if ids == 0:
                temporary.append(token)
                continue
            
            if tokens.events[ids].type_ =='Tempo':
                tempo = token
                continue
                

            if tokens.events[ids].type_ =='Bar':
                temporary.append(tempo)

                if len(li) + len(temporary) > max_tokenizer:

                    ## 바 하나에 악기가 너무 많이 들어가는 등 토큰의 갯수가 너무 많으면 temporary만으로도 max_tokenizer의 수를 넘어가므로 종료
                    if li:
                        tokenizer(li).dump_midi(f'{CHUNK_PATH}/{song[:-4]}/{num_mid}.mid')

                    else:
                        print("song is too long", song)

                        break
                    num_mid += 1
                    li=[]
                else:
                    li += temporary

                temporary = []
                
                temporary.append(token)
                continue
            
            temporary.append(token)
        
        temporary.append(tempo)
        if len(temporary) < max_tokenizer:
            tokenizer(temporary).dump_midi(f'{CHUNK_PATH}/{song[:-4]}/{num_mid}.mid')
        else:
            print(song)



 60%|██████    | 1327/2198 [00:11<00:10, 83.22it/s]  

song is too long Shock in my town.mid
Shock in my town.mid
song is too long Shock in my town.1.mid
Shock in my town.1.mid


 71%|███████   | 1551/2198 [05:21<05:15,  2.05it/s]  

song is too long Welcome Home (Sanitarium).1.mid
Welcome Home (Sanitarium).1.mid
song is too long Welcome Home (Sanitarium).2.mid
Welcome Home (Sanitarium).2.mid
song is too long Welcome Home (Sanitarium).mid
Welcome Home (Sanitarium).mid


 77%|███████▋  | 1693/2198 [08:19<09:40,  1.15s/it]  

...

 86%|████████▌ | 1882/2198 [14:09<18:09,  3.45s/it]  

song is too long I Put A Spell On You.1.mid
I Put A Spell On You.1.mid


 91%|█████████ | 2004/2198 [17:12<02:37,  1.23it/s]

song is too long Scream.1.mid
Scream.1.mid


100%|██████████| 2198/2198 [22:23<00:00,  1.64it/s]


### token 갯수 확인

In [21]:
CHUNK_PATH = '../max_token'
tokenizer_size = []

for p in tqdm(list(Path(f"../{CHUNK_PATH}").glob("**/*.mid"))):
    tokenizer_size.append(len(tokenizer(p).tokens))

max(tokenizer_size)

TypeError: 'int' object is not callable

In [31]:
tokenizer(['Bar_60', 'Position_0', 'Program_0', 'Pitch_38', 'Velocity_81', 'Duration_0.3.8', 'Program_32', 'Pitch_38', 'Velocity_81', 'Duration_0.3.8', 'Program_26', 'Pitch_50', 'Velocity_71', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_36', 'Velocity_65', 'Duration_1.0.8', 'Position_3', 'Program_0', 'Pitch_45', 'Velocity_69', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_69', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_59', 'Duration_0.1.8', 'Position_4', 'Program_0', 'Pitch_55', 'Velocity_91', 'Duration_0.3.8', 'Program_32', 'Pitch_55', 'Velocity_91', 'Duration_0.3.8', 'Program_26', 'Pitch_67', 'Velocity_81', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_51', 'Velocity_120', 'Duration_0.4.8', 'Position_7', 'Program_-1', 'PitchDrum_50', 'Velocity_45', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_60', 'Duration_0.1.8', 'Position_8', 'Program_0', 'Pitch_54', 'Velocity_84', 'Duration_0.3.8', 'Program_32', 'Pitch_54', 'Velocity_84', 'Duration_0.3.8', 'Program_26', 'Pitch_66', 'Velocity_74', 'Duration_0.3.8', 'Program_11', 'Pitch_64', 'Velocity_51', 'Duration_6.2.4', 'Program_11', 'Pitch_66', 'Velocity_51', 'Duration_6.2.4', 'Program_11', 'Pitch_69', 'Velocity_51', 'Duration_6.2.4', 'Program_-1', 'PitchDrum_36', 'Velocity_60', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_50', 'Velocity_55', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_70', 'Duration_0.1.8', 'Position_12', 'Program_0', 'Pitch_50', 'Velocity_69', 'Duration_0.3.8', 'Program_32', 'Pitch_50', 'Velocity_69', 'Duration_0.3.8', 'Program_26', 'Pitch_62', 'Velocity_59', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_51', 'Velocity_120', 'Duration_0.4.8', 'Position_15', 'Program_0', 'Pitch_45', 'Velocity_57', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_57', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_47', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_50', 'Velocity_45', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_60', 'Duration_0.1.8', 'Position_16', 'Program_0', 'Pitch_38', 'Velocity_69', 'Duration_0.3.8', 'Program_32', 'Pitch_38', 'Velocity_69', 'Duration_0.3.8', 'Program_26', 'Pitch_50', 'Velocity_59', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_36', 'Velocity_60', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_50', 'Velocity_55', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_70', 'Duration_0.1.8', 'Position_19', 'Program_0', 'Pitch_45', 'Velocity_54', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_54', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_44', 'Duration_0.1.8', 'Position_20', 'Program_0', 'Pitch_55', 'Velocity_87', 'Duration_0.3.8', 'Program_32', 'Pitch_55', 'Velocity_87', 'Duration_0.3.8', 'Program_26', 'Pitch_67', 'Velocity_77', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_51', 'Velocity_120', 'Duration_0.2.8', 'Position_23', 'Program_0', 'Pitch_54', 'Velocity_69', 'Duration_0.1.8', 'Program_32', 'Pitch_54', 'Velocity_69', 'Duration_0.1.8', 'Program_26', 'Pitch_66', 'Velocity_59', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_50', 'Velocity_45', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_60', 'Duration_0.1.8', 'Position_24', 'Program_-1', 'PitchDrum_36', 'Velocity_60', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_50', 'Velocity_55', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_64', 'Velocity_70', 'Duration_0.3.8', 'Position_27', 'Program_0', 'Pitch_50', 'Velocity_69', 'Duration_0.1.8', 'Program_32', 'Pitch_50', 'Velocity_69', 'Duration_0.1.8', 'Program_26', 'Pitch_62', 'Velocity_59', 'Duration_0.1.8', 'Position_28', 'Program_-1', 'PitchDrum_45', 'Velocity_80', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_46', 'Velocity_120', 'Duration_0.2.8', 'Position_31', 'Program_0', 'Pitch_45', 'Velocity_60', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_60', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_50', 'Duration_0.1.8', 'Tempo_93.0', 'Bar_61', 'Position_0', 'Program_0', 'Pitch_38', 'Velocity_81', 'Duration_0.3.8', 'Program_32', 'Pitch_38', 'Velocity_81', 'Duration_0.3.8', 'Program_26', 'Pitch_50', 'Velocity_71', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_36', 'Velocity_65', 'Duration_1.0.8', 'Position_3', 'Program_0', 'Pitch_45', 'Velocity_69', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_69', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_59', 'Duration_0.1.8', 'Position_4', 'Program_0', 'Pitch_55', 'Velocity_91', 'Duration_0.3.8', 'Program_32', 'Pitch_55', 'Velocity_91', 'Duration_0.3.8', 'Program_26', 'Pitch_67', 'Velocity_81', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_51', 'Velocity_120', 'Duration_0.4.8', 'Position_7', 'Program_-1', 'PitchDrum_50', 'Velocity_45', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_60', 'Duration_0.1.8', 'Position_8', 'Program_0', 'Pitch_54', 'Velocity_84', 'Duration_0.3.8', 'Program_32', 'Pitch_54', 'Velocity_84', 'Duration_0.3.8', 'Program_26', 'Pitch_66', 'Velocity_74', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_36', 'Velocity_60', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_50', 'Velocity_55', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_70', 'Duration_0.1.8', 'Position_12', 'Program_0', 'Pitch_50', 'Velocity_69', 'Duration_0.3.8', 'Program_32', 'Pitch_50', 'Velocity_69', 'Duration_0.3.8', 'Program_26', 'Pitch_62', 'Velocity_59', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_51', 'Velocity_120', 'Duration_0.4.8', 'Position_15', 'Program_0', 'Pitch_45', 'Velocity_57', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_57', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_47', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_50', 'Velocity_45', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_60', 'Duration_0.1.8', 'Position_16', 'Program_0', 'Pitch_38', 'Velocity_69', 'Duration_0.3.8', 'Program_32', 'Pitch_38', 'Velocity_69', 'Duration_0.3.8', 'Program_26', 'Pitch_50', 'Velocity_59', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_36', 'Velocity_60', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_50', 'Velocity_55', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_70', 'Duration_0.1.8', 'Position_19', 'Program_0', 'Pitch_45', 'Velocity_54', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_54', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_44', 'Duration_0.1.8', 'Position_20', 'Program_0', 'Pitch_55', 'Velocity_87', 'Duration_0.3.8', 'Program_32', 'Pitch_55', 'Velocity_87', 'Duration_0.3.8', 'Program_26', 'Pitch_67', 'Velocity_77', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_51', 'Velocity_120', 'Duration_0.2.8', 'Position_23', 'Program_0', 'Pitch_54', 'Velocity_69', 'Duration_0.1.8', 'Program_32', 'Pitch_54', 'Velocity_69', 'Duration_0.1.8', 'Program_26', 'Pitch_66', 'Velocity_59', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_50', 'Velocity_45', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_60', 'Duration_0.1.8', 'Position_24', 'Program_-1', 'PitchDrum_36', 'Velocity_60', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_50', 'Velocity_55', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_64', 'Velocity_70', 'Duration_0.3.8', 'Position_27', 'Program_0', 'Pitch_50', 'Velocity_69', 'Duration_0.1.8', 'Program_32', 'Pitch_50', 'Velocity_69', 'Duration_0.1.8', 'Program_26', 'Pitch_62', 'Velocity_59', 'Duration_0.1.8', 'Position_28', 'Program_-1', 'PitchDrum_45', 'Velocity_80', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_46', 'Velocity_120', 'Duration_0.2.8', 'Position_31', 'Program_0', 'Pitch_45', 'Velocity_60', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_60', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_50', 'Duration_0.1.8', 'Tempo_93.0', 'Bar_62', 'Position_0', 'Program_0', 'Pitch_38', 'Velocity_81', 'Duration_0.3.8', 'Program_32', 'Pitch_38', 'Velocity_81', 'Duration_0.3.8', 'Program_26', 'Pitch_50', 'Velocity_71', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_36', 'Velocity_65', 'Duration_1.0.8', 'Position_3', 'Program_0', 'Pitch_45', 'Velocity_69', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_69', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_59', 'Duration_0.1.8', 'Position_4', 'Program_0', 'Pitch_55', 'Velocity_91', 'Duration_0.3.8', 'Program_32', 'Pitch_55', 'Velocity_91', 'Duration_0.3.8', 'Program_26', 'Pitch_67', 'Velocity_81', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_51', 'Velocity_120', 'Duration_0.4.8', 'Position_7', 'Program_-1', 'PitchDrum_50', 'Velocity_45', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_60', 'Duration_0.1.8', 'Position_8', 'Program_0', 'Pitch_53', 'Velocity_84', 'Duration_0.3.8', 'Program_32', 'Pitch_53', 'Velocity_84', 'Duration_0.3.8', 'Program_26', 'Pitch_65', 'Velocity_74', 'Duration_0.3.8', 'Program_11', 'Pitch_64', 'Velocity_51', 'Duration_6.2.4', 'Program_11', 'Pitch_65', 'Velocity_51', 'Duration_6.2.4', 'Program_11', 'Pitch_69', 'Velocity_51', 'Duration_6.2.4', 'Program_-1', 'PitchDrum_36', 'Velocity_60', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_50', 'Velocity_55', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_70', 'Duration_0.1.8', 'Position_12', 'Program_0', 'Pitch_50', 'Velocity_69', 'Duration_0.3.8', 'Program_32', 'Pitch_50', 'Velocity_69', 'Duration_0.3.8', 'Program_26', 'Pitch_62', 'Velocity_59', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_51', 'Velocity_120', 'Duration_0.4.8', 'Position_15', 'Program_0', 'Pitch_45', 'Velocity_57', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_57', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_47', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_50', 'Velocity_45', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_60', 'Duration_0.1.8', 'Position_16', 'Program_0', 'Pitch_38', 'Velocity_69', 'Duration_0.3.8', 'Program_32', 'Pitch_38', 'Velocity_69', 'Duration_0.3.8', 'Program_26', 'Pitch_50', 'Velocity_59', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_36', 'Velocity_60', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_50', 'Velocity_55', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_70', 'Duration_0.1.8', 'Position_19', 'Program_0', 'Pitch_45', 'Velocity_54', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_54', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_44', 'Duration_0.1.8', 'Position_20', 'Program_0', 'Pitch_55', 'Velocity_87', 'Duration_0.3.8', 'Program_32', 'Pitch_55', 'Velocity_87', 'Duration_0.3.8', 'Program_26', 'Pitch_67', 'Velocity_77', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_51', 'Velocity_120', 'Duration_0.2.8', 'Position_23', 'Program_0', 'Pitch_53', 'Velocity_69', 'Duration_0.1.8', 'Program_32', 'Pitch_53', 'Velocity_69', 'Duration_0.1.8', 'Program_26', 'Pitch_65', 'Velocity_59', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_50', 'Velocity_45', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_60', 'Duration_0.1.8', 'Position_24', 'Program_-1', 'PitchDrum_36', 'Velocity_60', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_50', 'Velocity_55', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_64', 'Velocity_70', 'Duration_0.3.8', 'Position_27', 'Program_0', 'Pitch_50', 'Velocity_69', 'Duration_0.1.8', 'Program_32', 'Pitch_50', 'Velocity_69', 'Duration_0.1.8', 'Program_26', 'Pitch_62', 'Velocity_59', 'Duration_0.1.8', 'Position_28', 'Program_-1', 'PitchDrum_45', 'Velocity_80', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_46', 'Velocity_120', 'Duration_0.2.8', 'Position_31', 'Program_0', 'Pitch_45', 'Velocity_60', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_60', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_50', 'Duration_0.1.8', 'Tempo_93.0', 'Bar_63', 'Position_0', 'Program_0', 'Pitch_38', 'Velocity_81', 'Duration_0.3.8', 'Program_32', 'Pitch_38', 'Velocity_81', 'Duration_0.3.8', 'Program_26', 'Pitch_50', 'Velocity_71', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_36', 'Velocity_65', 'Duration_1.0.8', 'Position_3', 'Program_0', 'Pitch_45', 'Velocity_69', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_69', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_59', 'Duration_0.1.8', 'Position_4', 'Program_0', 'Pitch_55', 'Velocity_91', 'Duration_0.3.8', 'Program_32', 'Pitch_55', 'Velocity_91', 'Duration_0.3.8', 'Program_26', 'Pitch_67', 'Velocity_81', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_51', 'Velocity_120', 'Duration_0.4.8', 'Position_7', 'Program_-1', 'PitchDrum_50', 'Velocity_45', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_60', 'Duration_0.1.8', 'Position_8', 'Program_0', 'Pitch_53', 'Velocity_84', 'Duration_0.3.8', 'Program_32', 'Pitch_53', 'Velocity_84', 'Duration_0.3.8', 'Program_26', 'Pitch_65', 'Velocity_74', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_36', 'Velocity_60', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_50', 'Velocity_55', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_70', 'Duration_0.1.8', 'Position_12', 'Program_0', 'Pitch_50', 'Velocity_69', 'Duration_0.3.8', 'Program_32', 'Pitch_50', 'Velocity_69', 'Duration_0.3.8', 'Program_26', 'Pitch_62', 'Velocity_59', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_51', 'Velocity_120', 'Duration_0.4.8', 'Position_15', 'Program_0', 'Pitch_45', 'Velocity_57', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_57', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_47', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_50', 'Velocity_45', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_60', 'Duration_0.1.8', 'Position_16', 'Program_0', 'Pitch_38', 'Velocity_69', 'Duration_0.3.8', 'Program_32', 'Pitch_38', 'Velocity_69', 'Duration_0.3.8', 'Program_26', 'Pitch_50', 'Velocity_59', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_36', 'Velocity_60', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_50', 'Velocity_55', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_70', 'Duration_0.1.8', 'Position_19', 'Program_0', 'Pitch_45', 'Velocity_54', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_54', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_44', 'Duration_0.1.8', 'Position_20', 'Program_0', 'Pitch_55', 'Velocity_87', 'Duration_0.3.8', 'Program_32', 'Pitch_55', 'Velocity_87', 'Duration_0.3.8', 'Program_26', 'Pitch_67', 'Velocity_77', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_51', 'Velocity_120', 'Duration_0.2.8', 'Position_23', 'Program_0', 'Pitch_53', 'Velocity_69', 'Duration_0.1.8', 'Program_32', 'Pitch_53', 'Velocity_69', 'Duration_0.1.8', 'Program_26', 'Pitch_65', 'Velocity_59', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_50', 'Velocity_45', 'Duration_0.1.8', 'Program_-1', 'PitchDrum_64', 'Velocity_60', 'Duration_0.1.8', 'Position_24', 'Program_-1', 'PitchDrum_36', 'Velocity_60', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_50', 'Velocity_55', 'Duration_0.3.8', 'Program_-1', 'PitchDrum_64', 'Velocity_70', 'Duration_0.3.8', 'Position_27', 'Program_0', 'Pitch_50', 'Velocity_69', 'Duration_0.1.8', 'Program_32', 'Pitch_50', 'Velocity_69', 'Duration_0.1.8', 'Program_26', 'Pitch_62', 'Velocity_59', 'Duration_0.1.8', 'Position_28', 'Program_-1', 'PitchDrum_45', 'Velocity_100', 'Duration_1.0.8', 'Program_-1', 'PitchDrum_46', 'Velocity_95', 'Duration_0.2.8', 'Position_31', 'Program_0', 'Pitch_45', 'Velocity_60', 'Duration_0.1.8', 'Program_32', 'Pitch_45', 'Velocity_60', 'Duration_0.1.8', 'Program_26', 'Pitch_57', 'Velocity_50', 'Duration_0.1.8', 'Tempo_93.0'] )

Score(ttype=Tick, tpq=8, begin=0, end=132, tracks=5, notes=234, time_sig=1, key_sig=0, markers=0, lyrics=0)